In [4]:
import pandas as pd
from functools import reduce

# Read in the datasets
noc_regions_data = pd.read_csv("datasets/noc_regions.csv")
iso_data = pd.read_csv("datasets/regions_iso.csv")

In [2]:
# get the number of rows for each dataset
print("Number of rows in noc_regions.csv: ", noc_regions_data.shape[0])
print("Number of rows in regions_iso.csv: ", iso_data.shape[0])


Number of rows in noc_regions.csv:  245
Number of rows in regions_iso.csv:  271


In [6]:
# Merge the dataframes
merged_data = pd.merge(
    iso_data, noc_regions_data, left_on="name", right_on="region", how="left"
)

# Rename the columns for clarity
merged_data = merged_data.rename(
    columns={"NOC": "noc", "noc_full": "noc_full", "region": "region"}
)


In [7]:
merged_data.to_csv("datasets/merged_data.csv", index=False)

,name,iso_code,noc,noc_full,region,notes
0,Afghanistan,AFG,AFG,Islamic Republic of Afghanistan,Afghanistan,NaN
1,Åland Islands,ALA,NaN,NaN,NaN,NaN
2,Albania,ALB,ALB,Albania,Albania,NaN
3,Algeria,DZA,ALG,Algeria,Algeria,NaN
4,American Samoa,ASM,ASA,American Samoa,American Samoa,NaN
...,...,...,...,...,...,...
277,Yemen,YEM,YAR,North Yemen,Yemen,North Yemen
278,Yemen,YEM,YMD,South Yemen,Yemen,South Yemen
279,Zambia,ZMB,ZAM,Zambia,Zambia,NaN
280,Zimbabwe,ZWE,RHO,Zimbabwe,Zimbabwe,NaN
